# server connect

In [10]:
!pip install anvil-uplink
import anvil.server

anvil.server.connect("server_2BWTWXDY2V4ON7JLQ4HILKDO-QMTVJN4LDJALI4EQ")



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45228 sha256=821baf8a5bcc2422d4881d635e816c58bbaa7162b89d14bfe95751df4ba1783a
  Stored in directory: /root/.cache/pip/wheels/2e/7c/ad/d9c746276bf024d44296340869fcb169f1e5d80fb147351a57
Successfully built ws4py


Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Debug for azrafe7@gmail.com" as SERVER


# start

In [11]:
# constansts

START_WORD_LENGTHS = [4,5,6]

MIN_WORD_LENGTH = 3
MAX_WORD_LENGTH = 9

NUM_STEPS = 8
MIN_STEPS = 5


In [12]:
'''
Created on Feb 7, 2013

@author: olegs
'''

ROMAN_CONSTANTS = (
            ( "", "I", "II", "III", "IV", "V", "VI", "VII", "VIII", "IX" ),
            ( "", "X", "XX", "XXX", "XL", "L", "LX", "LXX", "LXXX", "XC" ),
            ( "", "C", "CC", "CCC", "CD", "D", "DC", "DCC", "DCCC", "CM" ),
            ( "", "M", "MM", "MMM", "",   "",  "-",  "",    "",     ""   ),
        )

ROMAN_SYMBOL_MAP = dict(I=1, V=5, X=10, L=50, C=100, D=500, M=1000)

CUTOFF = 4000
BIG_DEC = 2900
BIG_ROMAN = "MMCM"
ROMAN_NOUGHT = "nulla"

def digits(num):
    if num < 0:
        raise Exception('range error: negative numbers not supported')
    if num % 1 != 0.0:
        raise Exception('floating point numbers not supported')
    res = []
    while num > 0:
        res.append(num % 10)
        num //= 10
    return res

def toString(num, emptyZero=False):
    if num < CUTOFF:
        digitlist = digits(num)
        if digitlist:
            res = reversed([ ROMAN_CONSTANTS[order][digit] for order, digit in enumerate(digitlist) ])
            return "".join(res)
        else:
            return "" if emptyZero else ROMAN_NOUGHT
    else:
        if num % 1 != 0.0:
            raise Exception('floating point numbers not supported')
        # For numbers over or equal the CUTOFF, the remainder of division by 2900
        # is represented as above, prepended with the multiples of MMCM (2900 in Roman),
        # which guarantees no more than 3 repetitive Ms.
        return BIG_ROMAN * (num // BIG_DEC) + toString(num % BIG_DEC, emptyZero=True)

def parse(numeral):
    numeral = numeral.upper()
    result = 0

    lastVal = 0
    lastCount = 0
    subtraction = False
    for symbol in numeral[::-1]:
        value = ROMAN_SYMBOL_MAP.get(symbol)
        if not value:
            return False
        if lastVal == 0:
            lastCount = 1
            lastVal = value
        elif lastVal == value:
            lastCount += 1
            # exceptions
        else:
            result += (-1 if subtraction else 1) * lastVal * lastCount
            subtraction = lastVal > value
            lastCount = 1
            lastVal = value

    result = result + (-1 if subtraction else 1) * lastVal * lastCount
    return True

# loadDict

In [13]:
import requests
import time

dictionaryFileName = 'assets/dictionary_FULL_IT.txt'
dictionaryFileName = 'assets/Ita.txt'


def normalizeWord(word):
    return word.strip().upper()

LOAD_FROM_URL = True

t0 = time.time()

lines = []
if LOAD_FROM_URL:
    baseUrl = 'https://azrafe7.github.io/'
    f = requests.get(baseUrl + dictionaryFileName)
    lines = f.text.splitlines()
else:
    with open(dictionaryFileName) as f:
        lines = f.read().splitlines()

filteredWords = []
for line in lines:
    word = normalizeWord(line)
    length = len(word)
    if length >= MIN_WORD_LENGTH and length <= MAX_WORD_LENGTH and not parse(word):
        filteredWords.append(word)

print("Dictionary loaded in {0:.2f}s".format(time.time() - t0))

print('{0}/{1}'.format(len(filteredWords), len(lines)))

print(filteredWords[:10])

Dictionary loaded in 1.10s
401123/710993
['AALENIANA', 'AALENIANE', 'AALENIANI', 'AALENIANO', 'ABABUA', 'ABACA', 'ABACETI', 'ABACETO', 'ABACHI', 'ABACHISTA']


In [14]:

def wordsOfLength(lengths):
    return [word for word in filteredWords if len(word) in lengths]

len(list(wordsOfLength(START_WORD_LENGTHS)))


35094

In [15]:

def isAnagram(sourceWord, targetWord):
    if len(sourceWord) != len(targetWord):
        return False

    res = True

    sourceCharSet = list(sourceWord)
    targetCharSet = list(targetWord)

    #print(targetCharSet)
    for i, ch in enumerate(sourceWord):
        #print(ch, targetCharSet)
        if ch in targetCharSet:
            targetCharSet.remove(ch)
        else:
            res = False
            break

    return res


assert(isAnagram('', ''))
assert(not isAnagram('corto', 'trOca'))
assert(not isAnagram('c', ''))
assert(isAnagram('cortao', 'trocao'))

srcWord = 'ostra'
dstWord ='motri'

print('isAnagram("{0}", "{1}")'.format(srcWord, dstWord))
print(isAnagram(srcWord, dstWord))

isAnagram("ostra", "motri")
False


In [16]:
import time

def isAnagramEx(sourceWord, targetWord, maxDiffLength=0, strict=False):
    if maxDiffLength == 0:
        return isAnagram(sourceWord, targetWord)
    else:
        wordDiffLength = abs(len(sourceWord) - len(targetWord))
        if wordDiffLength > maxDiffLength:
            return False

    sourceCharSet = list(sourceWord)
    targetCharSet = list(targetWord)

    if len(sourceCharSet) > len(targetCharSet):
        sourceCharSet, targetCharSet = targetCharSet, sourceCharSet

    #print(targetCharSet, sourceCharSet)
    for ch in sourceCharSet:
        #print(ch, targetCharSet)
        if ch in targetCharSet:
            targetCharSet.remove(ch)

    #print(targetCharSet)
    targetCharSetLength = len(targetCharSet)
    #print(wordDiffLength, targetCharSet)
    isLooseAnagram = (targetCharSetLength <= maxDiffLength)
    res = (not strict and isLooseAnagram) or (strict and isLooseAnagram and wordDiffLength == maxDiffLength)
    return res


maxDiffLength=2

assert(isAnagramEx('', ''))
assert(not isAnagramEx('corto', 'trOca'))
assert(not isAnagramEx('c', ''))
assert(isAnagramEx('cortao', 'trocao'))
assert(isAnagramEx('cortao', 'trocaox', 1))
assert(not isAnagramEx('strappo', 'posta', 1))
assert(isAnagramEx('strappo', 'posta', 2))
assert(isAnagramEx('strappo', 'posta', 3))
assert(not isAnagramEx('strappo', 'posta', 3, strict=True))
assert(isAnagramEx('strappo', 'posta', 2, strict=True))
assert(isAnagramEx('LEVI', 'ABILE', 2,))
assert(not isAnagramEx('TUFFAI', 'ABACA', 2,))


srcWord = 'straccio'
dstWord ='crosta'

print('isAnagramEx("{0}", "{1}", {2})'.format(srcWord, dstWord, maxDiffLength))
print(isAnagramEx(srcWord, dstWord, maxDiffLength=maxDiffLength))

t0 = time.time()
for i in range(0):
  isAnagramEx('', '')
  not isAnagramEx('corto', 'trOca')
  not isAnagramEx('c', '')
  isAnagramEx('cortao', 'trocao')
  isAnagramEx('cortao', 'trocaox', 1)
  not isAnagramEx('strappo', 'posta', 1)
  isAnagramEx('strappo', 'posta', 2)
  isAnagramEx('strappo', 'posta', 3)
  not isAnagramEx('strappo', 'posta', 3, strict=True)
  isAnagramEx('strappo', 'posta', 2, strict=True)
  isAnagramEx('LEVI', 'ABILE', 2, )
  not isAnagramEx('TUFFAI', 'ABACA', 2, )
print(f'{(time.time() - t0):.4f}s')


isAnagramEx("straccio", "crosta", 2)
True
0.0002s


# cy_isAnagramEx2

In [17]:
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [18]:
%%cython
import cython
import time

@cython.boundscheck(False)
def _cy_isAnagramEx2(list sortedSourceChars, list sortedTargetChars, int sourceLength, int targetLength, int maxDiffLength=0, bint strict=False, bint failFast=False):
  if failFast and abs(sourceLength - targetLength) > maxDiffLength:
    return False, -1

  # if maxDiffLength == 0:
  #     return sortedSourceChars == sortedTargetChars
  # else:
  #   wordDiffLength = abs(sourceLength - targetLength)
  #   if wordDiffLength > maxDiffLength:
  #       return False

  cdef int i = 0
  cdef int j = 0
  cdef int diff = 0
  cdef int same = 0
  if sourceLength != 0 and targetLength != 0:
    while i < sourceLength and j < targetLength:
      sourceCh = sortedSourceChars[i]
      targetCh = sortedTargetChars[j]
      if sourceCh == targetCh:
        i += 1
        j += 1
        same += 1
      else:
        if sourceCh < targetCh:
          i += 1
        else:
          j += 1

  diff = max(sourceLength, targetLength) - same
  #print(sortedSourceChars, sortedTargetChars, diff, same)
  cdef bint isLooseAnagram = diff <= maxDiffLength
  cdef bint res = (not strict and isLooseAnagram) or (strict and isLooseAnagram and diff == maxDiffLength)
  return res, diff


cpdef cy_isAnagramEx2(str sourceWord, str targetWord, int maxDiffLength=0, bint strict=False, bint failFast=False):
  cdef list sortedSourceChars = sorted(sourceWord)
  cdef list sortedTargetChars = sorted(targetWord)

  # if maxDiffLength == 0:
  #     return sortedSourceChars == sortedTargetChars
  # else:


  cdef int sourceLength = len(sortedSourceChars)
  cdef int targetLength = len(sortedTargetChars)

  return _cy_isAnagramEx2(sortedSourceChars, sortedTargetChars, sourceLength, targetLength, maxDiffLength=maxDiffLength, strict=strict, failFast=failFast)

maxDiffLength = 2

assert (True, 0) == cy_isAnagramEx2('', '')
assert (False, 2) == cy_isAnagramEx2('corto', 'trOca')
assert (False, 4) == cy_isAnagramEx2('corto', 'trOcaul')
assert (False, 1) == cy_isAnagramEx2('c', '')
assert (True, 1) == cy_isAnagramEx2('cortao', 'trocaox', 1)
assert (True, 0) == cy_isAnagramEx2('cortao', 'trocao')
assert (False, 2) == cy_isAnagramEx2('strappo', 'posta', 1)
assert (True, 2) == cy_isAnagramEx2('strappo', 'posta', 2)
assert (True, 2) == cy_isAnagramEx2('strappo', 'posta', 3)
assert (False, 2) == cy_isAnagramEx2('strappo', 'posta', 3, strict=True)
assert (True, 2) == cy_isAnagramEx2('strappo', 'posta', 2, strict=True)
assert (True, 2) == cy_isAnagramEx2('LEVI', 'ABILE', 2, strict=False)
assert (False, 5) == cy_isAnagramEx2('TUFFAI', 'ABACA', 2, strict=False)
assert (False, 5) == cy_isAnagramEx2('TUFFAI', 'ABACA', 2, strict=True)
assert (True, 1) == cy_isAnagramEx2('AOPST', 'AOPSTU', 1, strict=False)
assert (True, 1) == cy_isAnagramEx2('AOPST', 'AOPSTU', 1, strict=True)

srcWord = 'straccio'
dstWord = 'crosta'

print('cy_isAnagramEx2("{0}", "{1}", {2})'.format(srcWord, dstWord, maxDiffLength))
print(cy_isAnagramEx2(srcWord, dstWord, maxDiffLength=maxDiffLength))

t0 = time.time()
for i in range(1000):
  cy_isAnagramEx2('', '')
  not cy_isAnagramEx2('corto', 'trOca')
  not cy_isAnagramEx2('c', '')
  cy_isAnagramEx2('cortao', 'trocao')
  cy_isAnagramEx2('cortao', 'trocaox', 1)
  not cy_isAnagramEx2('strappo', 'posta', 1)
  cy_isAnagramEx2('strappo', 'posta', 2)
  cy_isAnagramEx2('strappo', 'posta', 3)
  not cy_isAnagramEx2('strappo', 'posta', 3, strict=True)
  cy_isAnagramEx2('strappo', 'posta', 2, strict=True)
  cy_isAnagramEx2('LEVI', 'ABILE', 2, )
  not cy_isAnagramEx2('TUFFAI', 'ABACA', 2, )
print(f'{(time.time() - t0):.4f}s')


In [19]:

t0 = time.time()
for i in range(1000):
  cy_isAnagramEx2('', '')
  not cy_isAnagramEx2('corto', 'trOca')
  not cy_isAnagramEx2('c', '')
  cy_isAnagramEx2('cortao', 'trocao')
  cy_isAnagramEx2('cortao', 'trocaox', 1)
  not cy_isAnagramEx2('strappo', 'posta', 1)
  cy_isAnagramEx2('strappo', 'posta', 2)
  cy_isAnagramEx2('strappo', 'posta', 3)
  not cy_isAnagramEx2('strappo', 'posta', 3, strict=True)
  cy_isAnagramEx2('strappo', 'posta', 2, strict=True)
  cy_isAnagramEx2('LEVI', 'ABILE', 2, )
  not cy_isAnagramEx2('TUFFAI', 'ABACA', 2, )
print(f'{(time.time() - t0):.4f}s')

0.0175s


In [20]:
%%cython
import cython
from cpython cimport array
import array
import time
from libc.math cimport abs

@cython.boundscheck(False)
@cython.wraparound(False)
cdef _cy_isAnagramEx3(str sortedSourceChars, str sortedTargetChars, int sourceLength, int targetLength, int maxDiffLength=0, bint strict=False, bint failFast=False):
  if failFast and abs(sourceLength - targetLength) > maxDiffLength:
    return False, -1

  # if maxDiffLength == 0:
  #     return sortedSourceChars == sortedTargetChars
  # else:
  #   wordDiffLength = abs(sourceLength - targetLength)
  #   if wordDiffLength > maxDiffLength:
  #       return False

  cdef Py_ssize_t i = 0
  cdef Py_ssize_t j = 0
  cdef int diff = 0
  cdef int same = 0
  if sourceLength != 0 and targetLength != 0:
    while i < sourceLength and j < targetLength:
      sourceCh = sortedSourceChars[i]
      targetCh = sortedTargetChars[j]
      if sourceCh == targetCh:
        i += 1
        j += 1
        same += 1
      else:
        if sourceCh < targetCh:
          i += 1
        else:
          j += 1

  diff = max(sourceLength, targetLength) - same
  #print(sortedSourceChars, sortedTargetChars, diff, same)
  cdef bint isLooseAnagram = diff <= maxDiffLength
  cdef bint res = (not strict and isLooseAnagram) or (strict and isLooseAnagram and diff == maxDiffLength)
  return res, diff

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef cy_isAnagramEx3(str sourceWord, str targetWord, int maxDiffLength=0, bint strict=False, bint failFast=False):
  cdef str sortedSourceChars = ''.join(sorted(sourceWord))
  cdef str sortedTargetChars = ''.join(sorted(targetWord))

  cdef Py_ssize_t sourceLength = len(sortedSourceChars)
  cdef Py_ssize_t targetLength = len(sortedTargetChars)

  # cdef int[:] sourceArray = array.array('u', sortedSourceChars)
  # cdef int[:] targetArray = array.array('u', sortedTargetChars)
  # if maxDiffLength == 0:
  #     return sortedSourceChars == sortedTargetChars
  # else:


  if failFast and abs(sourceLength - targetLength) > maxDiffLength:
    return False, -1

  # if maxDiffLength == 0:
  #     return sortedSourceChars == sortedTargetChars
  # else:
  #   wordDiffLength = abs(sourceLength - targetLength)
  #   if wordDiffLength > maxDiffLength:
  #       return False

  cdef Py_ssize_t i = 0
  cdef Py_ssize_t j = 0
  cdef int diff = 0
  cdef int same = 0
  if sourceLength != 0 and targetLength != 0:
    while i < sourceLength and j < targetLength:
      sourceCh = sortedSourceChars[i]
      targetCh = sortedTargetChars[j]
      if sourceCh == targetCh:
        i += 1
        j += 1
        same += 1
      else:
        if sourceCh < targetCh:
          i += 1
        else:
          j += 1

  diff = max(sourceLength, targetLength) - same
  #print(sortedSourceChars, sortedTargetChars, diff, same)
  cdef bint isLooseAnagram = diff <= maxDiffLength
  cdef bint res = (not strict and isLooseAnagram) or (strict and isLooseAnagram and diff == maxDiffLength)


  return res, diff
  return _cy_isAnagramEx3(sortedSourceChars, sortedTargetChars, sourceLength, targetLength, maxDiffLength=maxDiffLength, strict=strict, failFast=failFast)


maxDiffLength = 2

assert (True, 0) == cy_isAnagramEx3('', '')
assert (False, 2) == cy_isAnagramEx3('corto', 'trOca')
assert (False, 4) == cy_isAnagramEx3('corto', 'trOcaul')
assert (False, 1) == cy_isAnagramEx3('c', '')
assert (True, 1) == cy_isAnagramEx3('cortao', 'trocaox', 1)
assert (True, 0) == cy_isAnagramEx3('cortao', 'trocao')
assert (False, 2) == cy_isAnagramEx3('strappo', 'posta', 1)
assert (True, 2) == cy_isAnagramEx3('strappo', 'posta', 2)
assert (True, 2) == cy_isAnagramEx3('strappo', 'posta', 3)
assert (False, 2) == cy_isAnagramEx3('strappo', 'posta', 3, strict=True)
assert (True, 2) == cy_isAnagramEx3('strappo', 'posta', 2, strict=True)
assert (True, 2) == cy_isAnagramEx3('LEVI', 'ABILE', 2, strict=False)
assert (False, 5) == cy_isAnagramEx3('TUFFAI', 'ABACA', 2, strict=False)
assert (False, 5) == cy_isAnagramEx3('TUFFAI', 'ABACA', 2, strict=True)
assert (True, 1) == cy_isAnagramEx3('AOPST', 'AOPSTU', 1, strict=False)
assert (True, 1) == cy_isAnagramEx3('AOPST', 'AOPSTU', 1, strict=True)

srcWord = 'straccio'
dstWord = 'crosta'

print('cy_isAnagramEx3("{0}", "{1}", {2})'.format(srcWord, dstWord, maxDiffLength))
print(cy_isAnagramEx3(srcWord, dstWord, maxDiffLength=maxDiffLength))

t0 = time.time()
for i in range(1000):
  cy_isAnagramEx3('', '')
  not cy_isAnagramEx3('corto', 'trOca')
  not cy_isAnagramEx3('c', '')
  cy_isAnagramEx3('cortao', 'trocao')
  cy_isAnagramEx3('cortao', 'trocaox', 1)
  not cy_isAnagramEx3('strappo', 'posta', 1)
  cy_isAnagramEx3('strappo', 'posta', 2)
  cy_isAnagramEx3('strappo', 'posta', 3)
  not cy_isAnagramEx3('strappo', 'posta', 3, strict=True)
  cy_isAnagramEx3('strappo', 'posta', 2, strict=True)
  cy_isAnagramEx3('LEVI', 'ABILE', 2, )
  not cy_isAnagramEx3('TUFFAI', 'ABACA', 2, )
print(f'{(time.time() - t0):.4f}s')


In [21]:

t0 = time.time()
for i in range(1000):
  cy_isAnagramEx3('', '')
  not cy_isAnagramEx3('corto', 'trOca')
  not cy_isAnagramEx3('c', '')
  cy_isAnagramEx3('cortao', 'trocao')
  cy_isAnagramEx3('cortao', 'trocaox', 1)
  not cy_isAnagramEx3('strappo', 'posta', 1)
  cy_isAnagramEx3('strappo', 'posta', 2)
  cy_isAnagramEx3('strappo', 'posta', 3)
  not cy_isAnagramEx3('strappo', 'posta', 3, strict=True)
  cy_isAnagramEx3('strappo', 'posta', 2, strict=True)
  cy_isAnagramEx3('LEVI', 'ABILE', 2, )
  not cy_isAnagramEx3('TUFFAI', 'ABACA', 2, )
print(f'{(time.time() - t0):.4f}s')

0.0252s


# isAnagramEx2

In [22]:
import time


def _isAnagramEx2(sortedSourceChars, sortedTargetChars, sourceLength, targetLength, maxDiffLength=0, strict=False, failFast=False):
  if failFast and abs(sourceLength - targetLength) > maxDiffLength:
    return False, -1

  # if maxDiffLength == 0:
  #     return sortedSourceChars == sortedTargetChars
  # else:
  #   wordDiffLength = abs(sourceLength - targetLength)
  #   if wordDiffLength > maxDiffLength:
  #       return False

  i = 0
  j = 0
  diff = 0
  same = 0
  if sourceLength != 0 and targetLength != 0:
    while i < sourceLength and j < targetLength:
      sourceCh = sortedSourceChars[i]
      targetCh = sortedTargetChars[j]
      if sourceCh == targetCh:
        i += 1
        j += 1
        same += 1
      else:
        if sourceCh < targetCh:
          i += 1
        else:
          j += 1

  diff = max(sourceLength, targetLength) - same
  #print(sortedSourceChars, sortedTargetChars, diff, same)
  isLooseAnagram = diff <= maxDiffLength
  res = (not strict and isLooseAnagram) or (strict and isLooseAnagram and diff == maxDiffLength)
  return res, diff


def isAnagramEx2(sourceWord, targetWord, maxDiffLength=0, strict=False, failFast=False):
  sortedSourceChars = sorted(sourceWord)
  sortedTargetChars = sorted(targetWord)

  # if maxDiffLength == 0:
  #     return sortedSourceChars == sortedTargetChars
  # else:


  sourceLength = len(sortedSourceChars)
  targetLength = len(sortedTargetChars)

  return _isAnagramEx2(sortedSourceChars, sortedTargetChars, sourceLength, targetLength, maxDiffLength=maxDiffLength, strict=strict, failFast=failFast)

maxDiffLength = 2

assert (True, 0) == isAnagramEx2('', '')
assert (False, 2) == isAnagramEx2('corto', 'trOca')
assert (False, 4) == isAnagramEx2('corto', 'trOcaul')
assert (False, 1) == isAnagramEx2('c', '')
assert (True, 1) == isAnagramEx2('cortao', 'trocaox', 1)
assert (True, 0) == isAnagramEx2('cortao', 'trocao')
assert (False, 2) == isAnagramEx2('strappo', 'posta', 1)
assert (True, 2) == isAnagramEx2('strappo', 'posta', 2)
assert (True, 2) == isAnagramEx2('strappo', 'posta', 3)
assert (False, 2) == isAnagramEx2('strappo', 'posta', 3, strict=True)
assert (True, 2) == isAnagramEx2('strappo', 'posta', 2, strict=True)
assert (True, 2) == isAnagramEx2('LEVI', 'ABILE', 2, strict=False)
assert (False, 5) == isAnagramEx2('TUFFAI', 'ABACA', 2, strict=False)
assert (False, 5) == isAnagramEx2('TUFFAI', 'ABACA', 2, strict=True)
assert (True, 1) == isAnagramEx2('AOPST', 'AOPSTU', 1, strict=False)
assert (True, 1) == isAnagramEx2('AOPST', 'AOPSTU', 1, strict=True)

srcWord = 'straccio'
dstWord = 'crosta'

print('isAnagramEx2("{0}", "{1}", {2})'.format(srcWord, dstWord, maxDiffLength))
print(isAnagramEx2(srcWord, dstWord, maxDiffLength=maxDiffLength))

t0 = time.time()
for i in range(0):
  isAnagramEx2('', '')
  not isAnagramEx2('corto', 'trOca')
  not isAnagramEx2('c', '')
  isAnagramEx2('cortao', 'trocao')
  isAnagramEx2('cortao', 'trocaox', 1)
  not isAnagramEx2('strappo', 'posta', 1)
  isAnagramEx2('strappo', 'posta', 2)
  isAnagramEx2('strappo', 'posta', 3)
  not isAnagramEx2('strappo', 'posta', 3, strict=True)
  isAnagramEx2('strappo', 'posta', 2, strict=True)
  isAnagramEx2('LEVI', 'ABILE', 2, )
  not isAnagramEx2('TUFFAI', 'ABACA', 2, )
print(f'{(time.time() - t0):.4f}s')


isAnagramEx2("straccio", "crosta", 2)
(True, 2)
0.0003s


In [23]:
import time
from collections import Counter

def isAnagram_Counter(sourceWord, targetWord, maxDiffLength=0, strict=False):
  sourceCounter = Counter(sourceWord)
  targetCounter = Counter(targetWord)

  # swap? (lengthier first!)
  if len(sourceWord) < len(targetWord):
    sourceCounter, targetCounter = targetCounter, sourceCounter

  diffCounter = sourceCounter - targetCounter

  diffSum = sum(diffCounter.values())

  isAnagram = (not strict and diffSum <= maxDiffLength) or (strict and diffSum == maxDiffLength)
  return isAnagram, diffSum

print("->", isAnagram_Counter('corto', 'trOca'))


assert (True, 0) == isAnagram_Counter('', '')
assert (False, 2) == isAnagram_Counter('corto', 'trOca')
assert (False, 4) == isAnagram_Counter('corto', 'trOcaul')
assert (False, 1) == isAnagram_Counter('c', '')
assert (True, 1) == isAnagram_Counter('cortao', 'trocaox', 1)
assert (True, 0) == isAnagram_Counter('cortao', 'trocao')
assert (False, 2) == isAnagram_Counter('strappo', 'posta', 1)
assert (True, 2) == isAnagram_Counter('strappo', 'posta', 2)
assert (True, 2) == isAnagram_Counter('strappo', 'posta', 3)
assert (False, 2) == isAnagram_Counter('strappo', 'posta', 3, strict=True)
assert (True, 2) == isAnagram_Counter('strappo', 'posta', 2, strict=True)
assert (True, 2) == isAnagram_Counter('LEVI', 'ABILE', 2, strict=False)
assert (False, 5) == isAnagram_Counter('TUFFAI', 'ABACA', 2, strict=False)
assert (False, 5) == isAnagram_Counter('TUFFAI', 'ABACA', 2, strict=True)
assert (True, 1) == isAnagram_Counter('AOPST', 'AOPSTU', 1, strict=False)
assert (True, 1) == isAnagram_Counter('AOPST', 'AOPSTU', 1, strict=True)
assert (True, 1) == isAnagram_Counter('BAVA', 'ABAVI', 1, strict=True)

t0 = time.time()
for i in range(0):
  isAnagram_Counter('', '')
  not isAnagram_Counter('corto', 'trOca')
  not isAnagram_Counter('c', '')
  isAnagram_Counter('cortao', 'trocao')
  isAnagram_Counter('cortao', 'trocaox', 1)
  not isAnagram_Counter('strappo', 'posta', 1)
  isAnagram_Counter('strappo', 'posta', 2)
  isAnagram_Counter('strappo', 'posta', 3)
  not isAnagram_Counter('strappo', 'posta', 3, strict=True)
  isAnagram_Counter('strappo', 'posta', 2, strict=True)
  isAnagram_Counter('LEVI', 'ABILE', 2, )
  not isAnagram_Counter('TUFFAI', 'ABACA', 2, )
print(f'{(time.time() - t0):.4f}s')


-> (False, 2)
0.0029s


In [24]:
import time
from copy import copy

class Counter2(dict):

  def __init__(self, word):
    self.word = word
    for ch in word:
      if ch not in self:
        self[ch] = 0
      self[ch] += 1
      # if ch not in self:
      #   self[ch] = word.count(ch)

  def __add__(self, otherCounter):
    newCounter = copy(self)
    for k, v in otherCounter.items():
      if k not in newCounter:
        newCounter[k] = 0
      newCounter[k] += v

    return newCounter

  def __sub__(self, otherCounter):
    newCounter = copy(self)
    for k, v in otherCounter.items():
      if k not in newCounter:
        newCounter[k] = 0
      else:
        newCounter[k] -= v
        if newCounter[k] <= 0:
          del newCounter[k]

    return newCounter

  def isAnagram(self, otherCounter, maxDiffLength=1, strict=False, failFast=False):
    sourceCounter, targetCounter = self, otherCounter
    sourceLength, targetLength = len(sourceCounter.word), len(targetCounter.word)
    if failFast and abs(targetLength - sourceLength) > maxDiffLength:
      return False, -1
    if sourceLength < targetLength:
      sourceCounter, targetCounter = targetCounter, sourceCounter
      sourceLength, targetLength = targetLength, sourceLength
    diffCounter = sourceCounter - targetCounter
    diffSum = sum(diffCounter.values())

    isAnagram = (not strict and diffSum <= maxDiffLength) or (strict and diffSum == maxDiffLength)
    return isAnagram, diffSum


def isAnagram_Counter2(sourceWord, targetWord, maxDiffLength=0, strict=False):
  sourceCounter = Counter2(sourceWord)
  targetCounter = Counter2(targetWord)

  # swap? (lengthier first!)
  if len(sourceWord) < len(targetWord):
    sourceCounter, targetCounter = targetCounter, sourceCounter

  diffCounter = sourceCounter - targetCounter

  diffSum = sum(diffCounter.values())

  isAnagram = (not strict and diffSum <= maxDiffLength) or (strict and diffSum == maxDiffLength)
  return isAnagram, diffSum

print("->", isAnagram_Counter2('corto', 'trOca'))

word1 = Counter2('corto')
word2 = Counter2('trOca')
display(word1)
display(word2)
display(word1 + word2)
display(word1 - word2)
display(Counter('corto') - Counter('trOca'))

assert (True, 0) == isAnagram_Counter2('', '')
assert (False, 2) == isAnagram_Counter2('corto', 'trOca')
assert (False, 4) == isAnagram_Counter2('corto', 'trOcaul')
assert (False, 1) == isAnagram_Counter2('c', '')
assert (True, 1) == isAnagram_Counter2('cortao', 'trocaox', 1)
assert (True, 0) == isAnagram_Counter2('cortao', 'trocao')
assert (False, 2) == isAnagram_Counter2('strappo', 'posta', 1)
assert (True, 2) == isAnagram_Counter2('strappo', 'posta', 2)
assert (True, 2) == isAnagram_Counter2('strappo', 'posta', 3)
assert (False, 2) == isAnagram_Counter2('strappo', 'posta', 3, strict=True)
assert (True, 2) == isAnagram_Counter2('strappo', 'posta', 2, strict=True)
assert (True, 2) == isAnagram_Counter2('LEVI', 'ABILE', 2, strict=False)
assert (False, 5) == isAnagram_Counter2('TUFFAI', 'ABACA', 2, strict=False)
assert (False, 5) == isAnagram_Counter2('TUFFAI', 'ABACA', 2, strict=True)
assert (True, 1) == isAnagram_Counter2('AOPST', 'AOPSTU', 1, strict=False)
assert (True, 1) == isAnagram_Counter2('AOPST', 'AOPSTU', 1, strict=True)
assert (True, 1) == isAnagram_Counter2('BAVA', 'ABAVI', 1, strict=True)
assert (True, 1) == Counter2('BAVA').isAnagram(Counter2('ABAVI'), maxDiffLength=1)

t0 = time.time()
for i in range(0):
  isAnagram_Counter2('', '')
  not isAnagram_Counter2('corto', 'trOca')
  not isAnagram_Counter2('c', '')
  isAnagram_Counter2('cortao', 'trocao')
  isAnagram_Counter2('cortao', 'trocaox', 1)
  not isAnagram_Counter2('strappo', 'posta', 1)
  isAnagram_Counter2('strappo', 'posta', 2)
  isAnagram_Counter2('strappo', 'posta', 3)
  not isAnagram_Counter2('strappo', 'posta', 3, strict=True)
  isAnagram_Counter2('strappo', 'posta', 2, strict=True)
  isAnagram_Counter2('LEVI', 'ABILE', 2, )
  not isAnagram_Counter2('TUFFAI', 'ABACA', 2, )
print(f'{(time.time() - t0):.4f}s')


-> (False, 2)


{'c': 1, 'o': 2, 'r': 1, 't': 1}

{'t': 1, 'r': 1, 'O': 1, 'c': 1, 'a': 1}

{'c': 2, 'o': 2, 'r': 2, 't': 2, 'O': 1, 'a': 1}

{'o': 2, 'O': 0, 'a': 0}

Counter({'o': 2})

0.0026s


# TreeNode

In [25]:
import random

def counterCompare(counterA, counterB):
  # counterA = Counter2(wordA)
  # counterB = Counter2(wordB)
  #isAnagram, diff = counterA.isAnagram(counterB, maxDiffLength=1, strict=True, failFast=True)
  isAnagram, diff = isAnagramEx2(counterA.word, counterB.word, maxDiffLength=1, strict=True)
  #print(counterA.word, counterB.word, (isAnagram, diff))
  return -1 if isAnagram else 1

def wordCompare(wordA, wordB):
  isAnagram, diff = isAnagramEx2(wordA, wordB, maxDiffLength=1, strict=True, failFast=True)
  #print(counterA.word, counterB.word, (isAnagram, diff))
  return -1 if isAnagram else 1

class TreeNode:
  def __init__(self, word):
    self.left = None
    self.right = None
    self.data = word

  def insert(self, word, compare=wordCompare):
    data = word

    y = None
    node = self
    i = 0
    while node is not None:
      i += 1
      if node.data:
        y = node
        comparison = compare(node.data, data)
        if comparison < 0:
          node = node.left
        else:
          node = node.right

    # if random.choice([0,1,2,4,5]) == 0:
    #   print(i)

    if y is None:
      self.data = data
    elif comparison < 0:
      y.left = TreeNode(word)
    else:
      y.right = TreeNode(word)

  def search(self, word, compare=wordCompare):
    results = []

    data = word

    y = None
    node = self
    while node is not None:
      if node.data:
        y = node
        comparison = compare(node.data, data)
        #print(comparison, node.data.word, data.word)
        if comparison < 0:
          results.append(node.data)
          node = node.left
        else:
          node = node.right

    return results


def printTree(node, level=0):
  if node != None:
    printTree(node.left, level + 1)
    print(' ' * 4 * level + '-> ' + str(node.word))
    printTree(node.right, level + 1)



t0 = time.time()
root = TreeNode('@')
shuffledWords = filteredWords[:]
random.shuffle(shuffledWords)
for i, word in enumerate(shuffledWords[:0]):
  root.insert(word)
  if i % 1000 == 0:
    print(i, word)
print(f'build took {(time.time() - t0):.2f}')

#printTree(root)

t0 = time.time()
word = shuffledWords[1]
res = root.search(word)
print(f'search {word} took {(time.time() - t0):.2f}')

display(res)


build took 0.42
search PIUMATONE took 0.00


[]

In [26]:
#print(shuffledWords[:100])
t0 = time.time()
word = random.choice(shuffledWords[:5000])
res = root.search(word)
print(f'search {word} took {(time.time() - t0):.2f}')

display(res)


search SUDEREBBE took 0.00


[]

# Trie

In [27]:
# http://stevehanov.ca/blog/index.php?id=114
# By Steve Hanov, 2011. Released to the public domain
import time
import sys

DICTIONARY = "/usr/share/dict/words";
TARGET = "SCORTA"
MAX_COST = 4

# Keep some interesting statistics
NodeCount = 0
WordCount = 0

# The Trie data structure keeps a set of words, organized with one node for
# each letter. Each node has a branch for each letter that may follow it in the
# set of words.
class TrieNode:
    def __init__(self):
        self.word = None
        self.children = {}

        global NodeCount
        NodeCount += 1

    def insert( self, word ):
        node = self
        for letter in word:
            if letter not in node.children:
                node.children[letter] = TrieNode()

            node = node.children[letter]

        node.word = word

# read dictionary file into a trie
start = time.time()

trie = TrieNode()
for word in filteredWords:
    WordCount += 1
    trie.insert( word )

end = time.time()
print("Build took %g s" % (end - start))

print("Read %d words into %d nodes" % (WordCount, NodeCount))

# The search function returns a list of all words that are less than the given
# maximum distance from the target word
def search( word, maxCost ):

    # build first row
    currentRow = range( len(word) + 1 )

    results = []

    # recursively search each branch of the trie
    for letter in trie.children:
        searchRecursive( trie.children[letter], letter, word, currentRow,
            results, maxCost )

    return results

# This recursive helper is used by the search function above. It assumes that
# the previousRow has been filled in already.
def searchRecursive( node, letter, word, previousRow, results, maxCost ):

    columns = len( word ) + 1
    currentRow = [ previousRow[0] + 1 ]

    # Build one row for the letter, with a column for each letter in the target
    # word, plus one for the empty string at column 0
    for column in range( 1, columns ):

        insertCost = currentRow[column - 1] + 1
        deleteCost = previousRow[column] + 1

        if word[column - 1] != letter:
            replaceCost = previousRow[ column - 1 ] + 1
        else:
            replaceCost = previousRow[ column - 1 ]

        currentRow.append( min( insertCost, deleteCost, replaceCost ) )

    # if the last entry in the row indicates the optimal cost is less than the
    # maximum cost, and there is a word in this trie node, then add it.
    if currentRow[-1] <= maxCost and node.word != None:
        results.append( (node.word, currentRow[-1] ) )

    # if any entries in the row are less than the maximum cost, then
    # recursively search each branch of the trie
    if min( currentRow ) <= maxCost:
        for letter in node.children:
            searchRecursive( node.children[letter], letter, word, currentRow,
                results, maxCost )

start = time.time()
results = search( TARGET, MAX_COST )
end = time.time()

#for result in results: print(result)

print("Search took %g s" % (end - start))



Build took 6.61241 s
Read 401123 words into 674739 nodes
Search took 1.3846 s


In [28]:
import random

startWord = random.choice(filteredWords)
print("START WORD:", startWord)

start = time.time()
results = search(startWord, 3)
end = time.time()

print("Search took %g s" % (end - start))

validEndWord = random.choice(results)

print("END WORD:", validEndWord)


START WORD: INSATURO
Search took 0.803543 s
END WORD: ('INSANIRA', 3)


# findAnagramsEx

In [42]:
def findAnagramsEx(wordList, word, maxDiffLength=1, strict=False):
    results = []
    wordLength = len(word)
    for entry in wordList:
        entryLength = len(entry)
        if word == entry or entryLength < wordLength - maxDiffLength or entryLength > wordLength + maxDiffLength:
            continue
        if cy_isAnagramEx2(entry, word, maxDiffLength=maxDiffLength, strict=strict):
            diffLength = abs(len(entry) - len(word))
            results.append((entry, diffLength))

    return results


In [44]:
import random

newStartWord = random.choice(filteredWords)
print("START WORD:", startWord)

anagramsDict = {0:[], 1:[], 2:[]}
randomFilteredWords = filteredWords[:]
random.shuffle(filteredWords[:])
randomFilteredWords = sorted(randomFilteredWords)[:100]

anagrams = findAnagramsEx(randomFilteredWords, newStartWord, maxDiffLength=1, strict=False)
for anagram in anagrams:
    anagramsDict[anagram[1]].append(anagram[0])

print(anagrams)
#print(anagramsDict[0][:10])

START WORD: BACAVO
[('AALENIANA', 0), ('AALENIANE', 0), ('AALENIANI', 0), ('AALENIANO', 0), ('ABACHISTA', 0), ('ABACHISTE', 0), ('ABACHISTI', 0), ('ABACISTA', 1), ('ABACISTI', 1), ('ABADESSA', 1), ('ABADESSE', 1), ('ABALIENA', 1), ('ABALIENAI', 0), ('ABALIENAR', 0), ('ABALIENI', 1), ('ABALIENO', 1), ('ABALIETA', 1), ('ABAPICALE', 0), ('ABAPICALI', 0), ('ABARICHE', 1), ('ABASICHE', 1), ('ABASSIALE', 0), ('ABASSIALI', 0), ('ABATESSA', 1), ('ABATESSE', 1), ('ABATJOUR', 1), ('ABATUCOLI', 0), ('ABATUCOLO', 0), ('ABAZIALE', 1), ('ABAZIALI', 1), ('ABBACACI', 1), ('ABBACAGLI', 0), ('ABBACALA', 1), ('ABBACALE', 1), ('ABBACALI', 1), ('ABBACALO', 1), ('ABBACAMI', 1), ('ABBACAMMO', 0), ('ABBACANDO', 0), ('ABBACANO', 1), ('ABBACANTE', 0), ('ABBACANTI', 0), ('ABBACARCI', 0), ('ABBACARE', 1), ('ABBACARLA', 0), ('ABBACARLE', 0), ('ABBACARLI', 0), ('ABBACARLO', 0), ('ABBACARMI', 0), ('ABBACARNE', 0), ('ABBACARSI', 0), ('ABBACARTI', 0), ('ABBACARVI', 0), ('ABBACASSE', 0), ('ABBACASSI', 0), ('ABBACASTE',

In [31]:
print(anagramsDict[1])

['AMATONE', 'AMATOVENE', 'AMMENTAVO', 'AUMENTAVO', 'CAVAMENTO', 'DENTAVAMO', 'EMANATIVO', 'EMANATO', 'EMANATOVI', 'EMANAVO', 'ENTRAVAMO', 'FOMENTAVA', 'LAMENTAVO', 'LAVAMENTO', 'LENTAVAMO', 'MANLEVATO', 'MANOVRATE', 'MANTECAVO', 'MANTOVANE', 'MEAVANO', 'MENTOVA', 'MENTOVALA', 'MENTOVATA', 'MENTOVAVA', 'MESTAVANO', 'MONCAVATE', 'MONDAVATE', 'MONTAVA', 'MONTAVATE', 'NESTAVAMO', 'NETTAVAMO', 'NORMAVATE', 'SENTAVAMO', 'SMONETAVA', 'TENDAVAMO', 'TENTAVAMO', 'TERNAVAMO', 'TRAMENAVO', 'TREMAVANO', 'TRENAVAMO', 'VAGAMENTO', 'VAMPATONE', 'VANEATO', 'VANEATOMI', 'VANTAMELO', 'VARAMENTO', 'VENTAVAMO']


# generateChain

In [45]:
import time

def generateChain(quiet=False, startWord=None, strict=False):
    startWords = wordsOfLength(START_WORD_LENGTHS)

    restart = True

    while restart:
        restart = False
        startWord = random.choice(startWords)
        print("START WORD:", startWord)

        steps = [startWord]

        prevStep = (startWord, 0)
        #self.info.text = ""
        for i in range(6):
            choice = random.choice(["anagram", "levenshtein"])
            if choice == "anagram":
                if not quiet: print("Finding anagrams...", end="")
                startTime = time.time()
                anagrams = findAnagramsEx(filteredWords, prevStep[0], maxDiffLength=1, strict=strict)
                if not quiet: print(" ({0:.2f})".format(time.time() - startTime))
                if len(anagrams) < 10 and i < MIN_STEPS:
                    restart = True
                    print(choice, len(anagrams))
                    break
            else:
                if not quiet: print("Finding levenshtein...", end="")
                startTime = time.time()
                results = search(prevStep[0], 1)
                if not quiet: print(" ({0:.2f})".format(time.time() - startTime))
                if len(results) < 10 and i < MIN_STEPS:
                    restart = True
                    print(choice, len(results))
                    break

            done = False
            while not done:
                if choice == "anagram":
                    nextStep = random.choice(anagrams)
                else:
                    nextStep = random.choice(results)

                if nextStep[0] not in steps:
                    steps.append(nextStep[0])
                    done = True

            resultsCount = len(results) if choice == "levenshtein" else len(anagrams)
            if not quiet: print(nextStep, resultsCount, choice)
            prevStep = nextStep

        print("restart", restart)
        print()

    endWord = steps[-1]
    print(steps[1:-1])
    print("END WORD:", endWord)
    return steps

t0 = time.time()
steps = generateChain(startWord=None, strict=True)
print("Elapsed {0:.2f}s".format(time.time() - t0))

startWord = steps[0]
endWord = steps[-1]

START WORD: DONARE
Finding anagrams... (0.21)
('TEPIDA', 0) 88662 anagram
Finding levenshtein... (0.00)
('TIEPIDA', 1) 10 levenshtein
Finding levenshtein... (0.00)
levenshtein 6
restart True

START WORD: SEDUCA
Finding levenshtein... (0.00)
('SEDUTA', 1) 10 levenshtein
Finding levenshtein... (0.00)
('SEDUTO', 1) 10 levenshtein
Finding anagrams... (0.21)
('CESTONE', 1) 88662 anagram
Finding levenshtein... (0.01)
('ESTONE', 1) 18 levenshtein
Finding levenshtein... (0.01)
('ESTINE', 1) 16 levenshtein
Finding levenshtein... (0.01)
('FESTINE', 1) 18 levenshtein
restart False

['SEDUTA', 'SEDUTO', 'CESTONE', 'ESTONE', 'ESTINE']
END WORD: FESTINE
Elapsed 0.53s


# checkValidChain

In [46]:
def isInDictionary(word):
    results = search(word, 0)
    return len(results) > 0

class WordStatus():
  UNCHECKED = 0
  VALID = 1
  INVALID = 2
  NOT_IN_DICT = 3

def checkValidChain(startWord, endWord, wordChain):
    prevWord = startWord
    isOk = True
    statuses = []
    print()
    for word in wordChain + [endWord]:
        inDictionary = isInDictionary(word)
        results = [t[0] for t in search(prevWord, 1)]
        anagrams = [t[0] for t in findAnagramsEx(filteredWords, prevWord, maxDiffLength=1, strict=False)]
        isWordOk = inDictionary and (word in results or word in anagrams)
        if not inDictionary:
            print("✖️", word)
            statuses.append(WordStatus.NOT_IN_DICT)
        else:
            print("✔️" if isWordOk else "❌", word)
            statuses.append(WordStatus.VALID if isWordOk else WordStatus.INVALID)

        if not isWordOk:
            isOk = False
        prevWord = word

    return isOk, statuses

print("START WORD:", startWord)
print("END WORD  :", endWord)

t0 = time.time()
checkValidChain(startWord, endWord, ["PAROLA"])
print("Elapsed {0:.2f}s".format(time.time() - t0))


START WORD: SEDUCA
END WORD  : FESTINE

✔️ PAROLA
✔️ FESTINE
Elapsed 0.48s


checkValidChain

In [47]:
def displayCheckValidChain(startWord, endWord, wordChain):
    print()
    print(startWord)
    ok, statuses = checkValidChain(startWord, endWord, wordChain)
    print(endWord)

    print()
    print("👍 WON" if ok else "👎 LOST")

    print()


In [48]:
# regenerate start/end words
t0 = time.time()
steps = generateChain(quiet=True, startWord='piccolo')
print("\nGenerate {0:.2f}s".format(time.time() - t0))

startWord = steps[0]
endWord = steps[-1]

print("START WORD:", startWord)
print("END WORD  :", endWord)

START WORD: SVOLLE
levenshtein 9
restart True

START WORD: UNCINO
levenshtein 6
restart True

START WORD: GLOTTE
levenshtein 7
restart True

START WORD: STRONG
levenshtein 2
restart True

START WORD: EQUESI
levenshtein 3
restart True

START WORD: RUBATO
restart False

['CUBATO', 'TUBATO', 'DETTETI', 'DETTEVI', 'QUERULE']
END WORD: QUERULI

Generate 2.31s
START WORD: RUBATO
END WORD  : QUERULI


In [49]:
wordChain = [
    'pantera',
    'pentola',
    'pantofola',
    'osteria',
    'ristorai',
    'piantala',
]
wordChain = [w.strip().upper() for w in wordChain]
#print("GUESS WORD CHAIN", wordChain)

t0 = time.time()
displayCheckValidChain(startWord, endWord, wordChain)
print("Validate {0:.2f}s".format(time.time() - t0))


RUBATO

✔️ PANTERA
✔️ PENTOLA
❌ PANTOFOLA
❌ OSTERIA
✔️ RISTORAI
✔️ PIANTALA
✔️ QUERULI
QUERULI

👎 LOST

Validate 3.96s


In [50]:
# STEPS

print("COMPUTED WORD CHAIN", steps)

COMPUTED WORD CHAIN ['RUBATO', 'CUBATO', 'TUBATO', 'DETTETI', 'DETTEVI', 'QUERULE', 'QUERULI']


In [51]:
# find in dictionary

findWord = 'muovi'

search(findWord.strip().upper(), 0)


[('MUOVI', 0)]

In [52]:
print([hex(ord(ch)) for ch in "✖️"])
print([hex(ord(ch)) for ch in "✔️"])
print([hex(ord(ch)) for ch in "❌"])
"👍"
hex(ord("👎"))

['0x2716', '0xfe0f']
['0x2714', '0xfe0f']
['0x274c']


'0x1f44e'

# server functions

In [53]:
@anvil.server.callable
def srvGenerateWordChain(quiet=True, startWord=None, strict=True):
  steps = generateChain(quiet=quiet, startWord=startWord, strict=strict)
  return steps

@anvil.server.callable
def srvValidateChain(startWord, endWord, wordChain):
  isOk, statuses = checkValidChain(startWord, endWord, wordChain)
  return isOk, statuses



# anagrams dict

In [54]:
t0 = time.time()

anagramsDict = {}
wordsByLength = {}
wordsBySortedAndLength = {}
for w in filteredWords[:]:
  sortedWord = ''.join(sorted(w))
  if sortedWord not in anagramsDict:
    anagramsDict[sortedWord] = set()
  anagramsDict[sortedWord].add(w)
  # moreAnagrams = findAnagramsEx(filteredWords, w, maxDiffLength=1, strict=True)
  # for item in moreAnagrams:
  #   anagramsDict[sortedWord].add(item[0])
  length = len(w)
  if length not in wordsByLength:
    wordsByLength[length] = set()
  wordsByLength[length].add(w)
  if w not in wordsBySortedAndLength:
    wordsBySortedAndLength[w] = [None, None, None]
  wordsBySortedAndLength[w][0] = sortedWord
  wordsBySortedAndLength[w][1] = length
  counter = Counter2(w)
  wordsBySortedAndLength[w][2] = counter


print(f'{(time.time() - t0):.2f}s')

display([item for item in random.choices(list(anagramsDict.items()), k=10)])
display([(k, list(v)[:5]) for k, v in wordsByLength.items()])
display([item for item in random.choices(list(wordsBySortedAndLength.items()), k=10)])


4.69s


[('AEINOPRRV', {'PROVINARE', 'PROVINERA', 'RIPROVANE'}),
 ('AAACFFIPR', {'PARAFFACI'}),
 ('AENV', {'NAVE', 'VANE', 'VENA'}),
 ('AEEGIMNTV', {'MINGEVATE'}),
 ('ABFIIINOV', {'INFOIBAVI'}),
 ('AEELPRT', {'PARTELE', 'PERLATE', 'PETARLE', 'REPLETA'}),
 ('AIMOPPZ', {'ZOPPAMI'}),
 ('AEEFPRSTU', {'STUPEFARE'}),
 ('AAIIMNSSV', {'VANISSIMA'}),
 ('AACGILQU', {'ACQUAGLI'})]

[(9, ['NASELLINI', 'REMEGGIAI', 'DIVELLERE', 'ZITTAMENE', 'UDITEVELO']),
 (6, ['ACUIVO', 'ODENTE', 'RIRESI', 'LEDILO', 'ACAICI']),
 (5, ['CRIMI', 'PEANA', 'MELME', 'PREMI', 'ANCOR']),
 (7, ['TRITERA', 'COLCALA', 'SACRALI', 'RIDURLA', 'RECRETA']),
 (8, ['PAGINATA', 'RAGUNATA', 'SCIFTALA', 'STORTANE', 'TRAVAGLI']),
 (4, ['LANI', 'TRAM', 'OSCA', 'IODA', 'PIGO'])]

[('ANNULLANO', ['AALLNNNOU', 9, {'A': 2, 'N': 3, 'U': 1, 'L': 2, 'O': 1}]),
 ('GABBANDO',
  ['AABBDGNO', 8, {'G': 1, 'A': 2, 'B': 2, 'N': 1, 'D': 1, 'O': 1}]),
 ('TERMITANI',
  ['AEIIMNRTT', 9, {'T': 2, 'E': 1, 'R': 1, 'M': 1, 'I': 2, 'A': 1, 'N': 1}]),
 ('SCOLATILE',
  ['ACEILLOST',
   9,
   {'S': 1, 'C': 1, 'O': 1, 'L': 2, 'A': 1, 'T': 1, 'I': 1, 'E': 1}]),
 ('STORNERAI',
  ['AEINORRST',
   9,
   {'S': 1, 'T': 1, 'O': 1, 'R': 2, 'N': 1, 'E': 1, 'A': 1, 'I': 1}]),
 ('PUNIRSELI',
  ['EIILNPRSU',
   9,
   {'P': 1, 'U': 1, 'N': 1, 'I': 2, 'R': 1, 'S': 1, 'E': 1, 'L': 1}]),
 ('TRIVIGINO',
  ['GIIINORTV', 9, {'T': 1, 'R': 1, 'I': 3, 'V': 1, 'G': 1, 'N': 1, 'O': 1}]),
 ('ADUNAVO', ['AADNOUV', 7, {'A': 2, 'D': 1, 'U': 1, 'N': 1, 'V': 1, 'O': 1}]),
 ('ULIMISSE',
  ['EIILMSSU', 8, {'U': 1, 'L': 1, 'I': 2, 'M': 1, 'S': 2, 'E': 1}]),
 ('BIVONGESE',
  ['BEEGINOSV',
   9,
   {'B': 1, 'I': 1, 'V': 1, 'O': 1, 'N': 1, 'G': 1, 'E': 2, 'S': 1}])]

In [55]:

# t0 = time.time()

# wordList = filteredWords[:21]
# for i, w in enumerate(wordList):
#   if i % 100 == 0:
#     print(f'{i:6}/{len(wordList)} {w}')
#   sortedWord, wordLength, sourceCounter = wordsBySortedAndLength[w]

#   candidates = []
#   targetLength = wordLength - 1
#   if targetLength in wordsByLength:
#     candidates = list(wordsByLength[targetLength])
#   targetLength = wordLength + 1
#   if targetLength in wordsByLength:
#     candidates += list(wordsByLength[targetLength])

#   for targetWord in candidates:
#     sortedTargetWord, targetLength, targetCounter = wordsBySortedAndLength[targetWord]
#     isAnagram, diff = _isAnagramEx2(sortedWord, sortedTargetWord, wordLength, targetLength, maxDiffLength=1, strict=True)

#     # if len(w) < len(targetWord):
#     #   sourceCounter, targetCounter = targetCounter, sourceCounter
#     # diffCounter = sourceCounter - targetCounter
#     # diffSum = sum(diffCounter.values())
#     # isAnagram = diffSum == 1

#     #sourceCounter.isAnagram(targetCounter, maxDiffLength = 1)

#     if isAnagram:
#       # print("->", targetWord)
#       anagramsDict[sortedWord].add(targetWord)
#       # display(anagramsDict[sortedWord])

# print(f'{(time.time() - t0):.2f}s')


In [56]:
word = 'ABAVI'
sortedWord, wordLength, sourceCounter = wordsBySortedAndLength[word]
display(anagramsDict[sortedWord])

_isAnagramEx2(list('BAVA'), list('ABAVI'), 4, 5, maxDiffLength=1, strict=True)


{'ABAVI', 'BIAVA'}

(False, 2)

In [57]:
print('LUCRAMELO'.__hash__())
print('LUCRAMELO'.__hash__())
print('lucramelo'.__hash__())

for k,v in wordsByLength.items():
  print(k, len(v))

#findAnagramsEx(filteredWords, 'TAVOLIERE', maxDiffLength=1, strict=True)

2972205265686595261
2972205265686595261
-5841338968531712146
9 200178
6 23116
5 9051
7 56496
8 109355
4 2927


# server wait

In [ ]:
anvil.server.wait_forever()

START WORD: STADIE
levenshtein 6
restart True

START WORD: VOCANO
levenshtein 4
restart True

START WORD: WHIG
levenshtein 5
restart True

START WORD: CRETA
levenshtein 8
restart True

START WORD: ELODEA
levenshtein 3
restart True

START WORD: ERUDIR
levenshtein 6
restart True

START WORD: SIPAI
levenshtein 8
restart True

START WORD: ORRIDO
levenshtein 6
restart True

START WORD: VILITI
levenshtein 3
restart True

START WORD: MAQUIS
levenshtein 9
restart True

START WORD: COLGO
restart False

['DUCARE', 'ONERA', 'SONERA', 'ZONERA', 'PESCINE']
END WORD: MUSATOLI
START WORD: BANK
levenshtein 8
restart True

START WORD: RUMAR
levenshtein 2
restart True

START WORD: TANDEM
levenshtein 6
restart True

START WORD: ITALI
levenshtein 2
restart True

START WORD: LIBATA
restart False

['LOBATA', 'ARAMITI', 'RIPIENAR', 'LIGIALE', 'MORATIVI']
END WORD: BIVACCAVA
START WORD: VEPSO
levenshtein 6
restart True

START WORD: SBASIR
levenshtein 9
restart True

START WORD: DICALO
restart False

['RICALO'